# Portfolio calculaltor

In [1]:
import yfinance as yf
import pandas as pd
from functions import process_stock_data

Add the tickers of your portfolio

In [2]:
tickers = ["NOK"] 
fileName = tickers


Create an empty DataFrame to store the information for all tickers

In [3]:
df = pd.DataFrame()

Loop through each ticker and retrieve the information

In [4]:
for symbol in tickers:
    stock = yf.Ticker(ticker=symbol)
    stock_info = stock.get_info()

    # Filter out unwanted values from the dictionary and append to the DataFrame
    wanted_values = ['website', 'longName', 'industry', 'sector', 'fullTimeEmployees', 'auditRisk', 'boardRisk', 'previousClose', 'open', 'dayLow', 'dayHigh', 'dividendRate', 'dividendYield', 'payoutRatio', 'trailingPE', 'forwardPE', 'marketCap', 'priceToSalesTrailing12Months', 'enterpriseValue', 'profitMargins', 'priceToBook', 'trailingEps', 'freeCashflow', 'Growth Estimates']
    # Array for rounded values
    round_values = ['marketCap', 'enterpriseValue', 'freeCashflow']
    # converted values
    convert_to_euros = ['previousClose', 'marketCap', 'open', 'dayLow', 'dayHigh', 'dividendRate', 'freeCashflow'] 

    df_temp = pd.DataFrame.from_dict(data=process_stock_data(stock_info, wanted_values, round_values, convert_to_euros, symbol), orient='columns')
    df = pd.concat([df, df_temp], ignore_index=True) 

Decorate the columns

In [5]:
styles = [
    {'selector': 'th', 'props': [('background-color', 'red'), ('color', 'white'), ('font-weight', 'bold'), ('text-align', 'center')]},
    {'selector': 'tr:nth-child(odd)', 'props': [('background-color', 'pink'), ('text-align', 'center'), ('font-weight', 'bold')]},
    {'selector': 'tr:nth-child(even)', 'props': [('background-color', 'white'), ('color', 'black'), ('text-align', 'center'), ('font-weight', 'bold')]},
]
df_styled = df.style.set_table_styles(styles)

# Display the styled DataFrame
df_styled

,website,industry,sector,fullTimeEmployees,auditRisk,boardRisk,previousClose,open,dayLow,dayHigh,dividendRate,dividendYield,payoutRatio,trailingPE,forwardPE,marketCap,priceToSalesTrailing12Months,enterpriseValue,profitMargins,priceToBook,trailingEps,longName,freeCashflow,symbol,Five year growth,Intrinsic value,safety margin,Acceptable buying price,Buy/Sell
0,https://www.nokia.com,Communication Equipment,Technology,86896,1,1,3.770000,3.780000,3.760000,3.810000,0.110000,0.028300,0.122000,4.810126,8.260869,21069.707264,0.834114,19414.568960,0.164290,0.995285,0.790000,Nokia Oyj,-474.500000,NOK,6.720000,6.031274,0.650000,3.920328,Buy


Save the DataFrame to a CSV file with a filename based on the tickers

In [ ]:
csv_file = '_'.join(tickers)
df.to_csv(f'{tickers}.csv', index=False)